erstes Modell, das trainiert wird

accuracy liegt bei 0  -> irgendwo Fehler im Aufbau bzw. Zuordnung von Label und Daten

Label müssen als Zahlen angegeben werden, nicht als Buchstaben

In [1]:
import tensorflow as tf
import os
import json
import tempfile

In [ ]:
# normalize the RGB values to be between 0 and 1
train_images = train_images / 255.0
test_images = test_images / 255.0

train dataset

In [2]:
"""
  geht jeden Ordner in train durch, speichert jedes Bild als tensor
  und ordnet jedem Bild im Unterordner den Namen des Ordners als Label zu
"""
from natsort import natsorted
images_path = "./train/"
images_dirs = os.listdir(images_path)   # folder of single buchstabe   -> a,b,c,d,e,f,g,h,i,j,k, ...
tensor_img = []   
tensor_l = []

# get images as tensors
for folder in natsorted(os.listdir(images_path)):
    label = folder   # a,b,c,d, ...
    print(label)

    for file in os.listdir(os.path.join(images_path, folder)):
        path = os.path.join(images_path, folder, file)

        if file.lower().endswith(('.jpeg', '.jpg')):
            image = tf.io.read_file(path)
            tensor_image = tf.io.decode_image(image, channels=3, dtype=tf.dtypes.float32)
            tensor_image = tf.image.resize(tensor_image, [256, 256])
            tensor_img.append(tensor_image)
            tensor_l.append(label)
            tensor_l = list(map(int,tensor_l))
        else: 
            print("No images in folder.")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


In [3]:
"""
 Test, ob label und Bild richtig zugeordnet
"""
#print(f'tensor_image: {tensor_img[7599]}')
print(f'tensor_label: {tensor_l[7599]}')

tensor_label: 26


train data set as tensor dataset

In [4]:
"""
  Erstellen des tensorflow Train Datasets
  Überprüfen, ob Bild und Label richtig
"""
train_dataset = tf.data.Dataset.from_tensor_slices((tensor_img, tensor_l))

element_index = 422
desired_element = train_dataset.skip(element_index).take(1)
for image, label in desired_element:
# Hier kannst du überprüfen, ob das Bild und das Label korrekt zusammengefügt wurden
    print("Label:", label)

Label: tf.Tensor(2, shape=(), dtype=int32)


In [5]:
"""
 Vermischen erfolgreich, da unterschiedliches Label
"""
train_dataset = train_dataset.shuffle(buffer_size=len(tensor_img))

element_index = 422
desired_element = train_dataset.skip(element_index).take(1)
for image, label in desired_element:
# Hier kannst du überprüfen, ob das Bild und das Label korrekt zusammengefügt wurden
    print("Label:", label)

Label: tf.Tensor(21, shape=(), dtype=int32)


test data

In [6]:
from natsort import natsorted
images_path = "./test/"
images_dirs = os.listdir(images_path)   # folder of single buchstabe   -> a,b,c,d,e,f,g,h,i,j,k, ...
tensor_img = []   
tensor_l = []

# get images as tensors
for folder in natsorted(os.listdir(images_path)):
    label = folder   # a,b,c,d, ...
    print(label)

    for file in os.listdir(os.path.join(images_path, folder)):
        path = os.path.join(images_path, folder, file)

        if file.lower().endswith(('.jpeg', '.jpg')):
            image = tf.io.read_file(path)
            tensor_image = tf.io.decode_image(image, channels=3, dtype=tf.dtypes.float32)
            tensor_image = tf.image.resize(tensor_image, [256, 256])
            tensor_img.append(tensor_image)
            tensor_l.append(label)
            tensor_l = list(map(int,tensor_l))
        else: 
            print("No images in folder.")


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26


In [7]:
#print(f'tensor_image: {tensor_img[5]}')
print(f'tensor_label: {tensor_l[5]}')

tensor_label: 1


In [8]:
test_dataset = tf.data.Dataset.from_tensor_slices((tensor_img, tensor_l))

element_index = 2
desired_element = test_dataset.skip(element_index).take(1)
for image, label in desired_element:
# Hier kannst du überprüfen, ob das Bild und das Label korrekt zusammengefügt wurden
    print("Label:", label)

Label: tf.Tensor(1, shape=(), dtype=int32)


In [9]:
test_dataset = test_dataset.shuffle(buffer_size=len(tensor_img))

element_index = 2
desired_element = test_dataset.skip(element_index).take(1)
for image, label in desired_element:
# Hier kannst du überprüfen, ob das Bild und das Label korrekt zusammengefügt wurden
    print("Label:", label)

Label: tf.Tensor(4, shape=(), dtype=int32)


In [10]:
train_dataset = train_dataset.batch(10)  # Trainingssatz wird in Gruppen von jeweils 10 Elementen aufgeteilt  -> kleinere Menge einfacher für Modell
test_dataset = test_dataset.batch(10)

In [13]:
model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(256,256,3)),  # 32 Feature-Maps, Größe des Filters, Aktivierungsfunktion, Bildgröße
        tf.keras.layers.MaxPooling2D((2,2)),   # Größe des Pooling-Fensters
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D((2,2)),
        tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10, activation='relu'), # Anzahl Neuronen
        tf.keras.layers.Dense(5),
        #tf.keras.layers.Reshape((4,2), input_shape=(None, 8))
    ])

model.compile(optimizer='adam',
                   loss= 'mean_squared_error', 
                   metrics=['accuracy'])
    # 'mean_squared_error'

    # train model
model.fit(train_dataset, epochs=10)
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"test loss: {test_loss}")
print('\nTest accuracy: {}'.format(test_accuracy))

Epoch 1/10
761/761 [==============================] - 328s 430ms/step - loss: 34.6251 - accuracy: 0.0232
Epoch 2/10
761/761 [==============================] - 328s 431ms/step - loss: 13.2551 - accuracy: 5.2625e-04
Epoch 3/10
761/761 [==============================] - 329s 432ms/step - loss: 8.2444 - accuracy: 1.3156e-04
Epoch 4/10
761/761 [==============================] - 331s 435ms/step - loss: 5.1161 - accuracy: 0.0000e+00
Epoch 5/10
761/761 [==============================] - 329s 433ms/step - loss: 3.4919 - accuracy: 0.0000e+00
Epoch 6/10
761/761 [==============================] - 328s 431ms/step - loss: 2.3798 - accuracy: 0.0000e+00
Epoch 7/10
761/761 [==============================] - 326s 429ms/step - loss: 1.8643 - accuracy: 0.0000e+00
Epoch 8/10
761/761 [==============================] - 325s 427ms/step - loss: 1.4337 - accuracy: 0.0000e+00
Epoch 9/10
761/761 [==============================] - 324s 426ms/step - loss: 1.4182 - accuracy: 0.0000e+00
Epoch 10/10
58/58 [===========

verworfen

In [31]:
# Gruppierung des Datasets nach Labels  -> key = labels, 
grouped_dataset = tf_dataset.group_by_window(key_func=lambda x: x[1], reduce_func=lambda key, ds: ds.batch(20),window_size=-1)

# Auswahl einer gleichen Anzahl von Daten für jedes Label
train_size_per_label = 170
full_dataset = grouped_dataset.flat_map(lambda key, ds: ds.take(train_size_per_label))

# Mischen des balancierten Datasets
train_dataset = full_dataset.shuffle(buffer_size=1000)

# Iteration über das train_dataset und Anzeigen der Labels
for img, label in train_dataset:
    print("Label:", label.numpy())

TypeError: <lambda>() takes 1 positional argument but 2 were given

In [6]:
train_size = int(0.7 * 7601) # hart codiert 
val_size = int(0.15 * 7601)
test_size = int(0.15 * 7601)

full_dataset = tf_dataset.shuffle(20)
train_dataset = full_dataset.take(train_size)
test_dataset = full_dataset.skip(train_size)
val_dataset = test_dataset.skip(test_size)
test_dataset = test_dataset.take(test_size)

In [8]:
index = 7599
element = full_dataset.skip(element_index).take(1)
for image, label in element:
    print("Label: ", label)

Label:  tf.Tensor(1, shape=(), dtype=int32)


In [13]:
dataset_size = tf.data.experimental.cardinality(train_dataset).numpy()

print("Die Größe des Datasets beträgt:", dataset_size)

Die Größe des Datasets beträgt: 5320


In [26]:
for img, label in train_dataset:
    print("Label:", label.numpy())

Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
Label: 1
L

In [22]:
index = 3000
element = train_dataset.skip(element_index).take(1)
for image, label in element:
    print("Label: ", label)

Label:  tf.Tensor(2, shape=(), dtype=int32)


In [25]:
index = 40
element = test_dataset.skip(element_index).take(1)
for image, label in element:
    print("Label: ", label)

Label:  tf.Tensor(20, shape=(), dtype=int32)
